In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
import prometheux_chain as px

px_TOKEN = "dev-token"
os.environ["px_TOKEN"] = px_TOKEN

#### Project functions

In [ ]:
px.cleanup_projects()

In [ ]:
project_id = px.save_project(project_name="test_project")
project_id

In [ ]:
px.list_projects()

In [ ]:
project = px.load_project(project_id=project_id)
project

#### Data functions

In [ ]:
#px.cleanup_sources()

In [ ]:
db = px.data.database.Database(
    database_type="csv",
    username="",
    password="",
    host=os.getcwd() + "/../resources/ownership",
    port=0000,
    database_name="ownership.csv",
    options={} #{"delimiter": "\t"}
)
px.connect_sources(database_payload=db)

In [ ]:
px.list_sources()

In [ ]:
px.infer_schema(database=db, add_bind=True, add_model=False)

#### Concept functions

In [ ]:
#px.cleanup_concepts(project_id=project_id)

In [ ]:
concept_logic = """
ownership(Owner,Company,Shares) :- ownership_csv(Owner,Company,Shares).
@output("ownership").
"""

px.save_concept(project_id=project_id, concept_logic=concept_logic)

concept_logic = """
owner(Owner) :- ownership(Owner,Company,Shares).
@output("owner").
"""

px.save_concept(project_id=project_id, concept_logic=concept_logic)

concept_logic = """
company("Apple", "Redwood City, CA").
company("Google", "Mountain View, CA").
company("Microsoft", "Redmond, WA").
company("Amazon", "Seattle, WA").
company("Facebook", "Menlo Park, CA").
company("Twitter", "San Francisco, CA").
company("LinkedIn", "Sunnyvale, CA").
company("Instagram", "Menlo Park, CA").

location(Location) :- company(_,Location).

@output("location").
"""

px.save_concept(project_id=project_id, concept_logic=concept_logic)

In [ ]:
px.run_concept(project_id=project_id, concept_name="ownership", persist_outputs=True)

In [ ]:
px.run_concept(project_id=project_id, concept_name="location", persist_outputs=False)

In [ ]:
python_script = """
import json

def main():
    # Simple calculations
    numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    # Calculate basic statistics
    total = sum(numbers)
    average = total / len(numbers)
    maximum = max(numbers)
    minimum = min(numbers)
    
    # Create results as a list of dictionaries (tabular format)
    results = [
        {
            "Numbers": numbers,
            "Total": total,
            "Average": round(average, 2),
            "Maximum": maximum,
            "Minimum": minimum,
            "Count": len(numbers)
        }
    ]
    
    return results
"""

px.save_concept(project_id=project_id, 
                concept_logic="@output(\"test_python\").\ntest_python(Numbers,Total,Average,Maximum,Minimum,Count) :- python:run(\"python_script\").",
                python_scripts={
                     "python_script": python_script
                 })

px.run_concept(project_id=project_id, 
                 concept_name="test_python")

In [ ]:
px.list_concepts(project_id=project_id)

##### KG functions

In [ ]:
px.save_kg(project_id=project_id, concepts=["owner", "location"])

In [ ]:
px.load_kg(project_id=project_id)

#### Copy project function

In [ ]:
px.copy_project(project_id=project_id, workspace_id="workspace_id", new_project_name="copied_project")

In [ ]:
px.list_projects()

#### GraphRAG function

In [ ]:
graph_rag_result = px.graph_rag(project_id=project_id,
                                  question="This is the question", 
                                  rag_concepts=[{"concept": "ownership", "field_to_embed": "Owner"}],
                                  graph_selected_concepts=["owner"])

graph_rag_result['graph_rag_results']